In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

True 2.1.1+cu118


In [2]:
from src.api import Trainer  
app = Trainer.initialize(stage = 0 , resume = 1 , checkname= 1)
app.go()


Use device name: NVIDIA GeForce RTX 4090


24-07-03 23:28:59|MOD:display     |: Model Specifics:
24-07-03 23:28:59|MOD:display     |: Start Process [Data] at Wed Jul  3 23:28:59 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_avg!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
Callback : DetailedAlphaAnalysis(which_alpha=avg) , record and concat each model to Alpha model instance
{'random_seed': None,
 'model_name': 'gru_avg',
 'model_m

24-07-03 23:29:01|MOD:display     |: Finish Process [Data], Cost 2.0 Secs
24-07-03 23:29:01|MOD:display     |: Start Process [Fit] at Wed Jul  3 23:29:01 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


...

24-07-04 05:55:08|MOD:display     |: Finish Process [Test], Cost 90.8 Secs
24-07-04 05:55:16|MOD:test        |: Performing Factor and FMP test!


PerfManager calc Finished!
PerfManager plot Finished!
Group optimization of 3 alphas , 3 benchmarks , 2 lags , 363 dates , (6534 opts) start!
Done Optimize    0th [best.csi300      ] at 20170104 , time cost (ms) : {'parse_input': 19.9, 'solve': 102.03, 'output': 18.13}
Done Optimize   50th [swalast.csi1000  ] at 20170118 , time cost (ms) : {'parse_input': 19.94, 'solve': 109.46, 'output': 20.85}
Done Optimize  100th [swabest.csi500.1 ] at 20170215 , time cost (ms) : {'parse_input': 19.28, 'solve': 107.61, 'output': 19.96}
Done Optimize  150th [swabest.csi300   ] at 20170308 , time cost (ms) : {'parse_input': 18.68, 'solve': 130.65, 'output': 19.44}
Done Optimize  200th [best.csi1000     ] at 20170329 , time cost (ms) : {'parse_input': 18.86, 'solve': 117.14, 'output': 19.91}
Done Optimize  250th [swalast.csi500.1 ] at 20170414 , time cost (ms) : {'parse_input': 19.35, 'solve': 125.56, 'output': 19.97}
Done Optimize  300th [swalast.csi300   ] at 20170508 , time cost (ms) : {'parse_input

,factor_name,benchmark,start,end,pf,bm,excess,annualized,mdd,te,ir,calmar,turnover,mdd_period
1,best,csi300,20170104,20240628,175.90%,21.07%,154.83%,11.45%,4.65%,4.45%,2.571,2.465,146.006,20210623-20210901
2,best,csi500,20170104,20240628,181.43%,-14.55%,195.98%,17.17%,8.26%,6.60%,2.603,2.079,146.148,20231225-20240206
0,best,csi1000,20170104,20240628,170.24%,-39.46%,209.70%,21.81%,14.22%,6.83%,3.195,1.534,146.256,20210429-20210908
4,swabest,csi300,20170104,20240628,180.11%,21.07%,159.04%,11.71%,4.56%,4.41%,2.658,2.568,146.020,20210623-20210908
5,swabest,csi500,20170104,20240628,201.82%,-14.55%,216.37%,18.26%,9.44%,6.93%,2.636,1.933,146.143,20240109-20240206
3,swabest,csi1000,20170104,20240628,200.71%,-39.46%,240.17%,23.61%,13.40%,7.07%,3.339,1.763,146.239,20210429-20210908
7,swalast,csi300,20170104,20240628,182.02%,21.07%,160.95%,11.82%,5.10%,4.38%,2.698,2.318,146.028,20231225-20240628
8,swalast,csi500,20170104,20240628,197.56%,-14.55%,212.11%,18.09%,10.43%,6.85%,2.643,1.735,146.158,20210601-20210908
6,swalast,csi1000,20170104,20240628,177.27%,-39.46%,216.73%,22.29%,16.09%,6.95%,3.207,1.385,146.234,20210429-20210901


24-07-04 06:37:08|MOD:time        |: Main Process Finished! Cost 7 Hours 8 Minutes 8.3 Seconds


Analytic datas are saved to /home/mengkjin/Workspace/learndl/model/gru_avg/detailed_analysis/data.xlsx
Analytic plots are saved to /home/mengkjin/Workspace/learndl/model/gru_avg/detailed_analysis/plot.pdf


In [ ]:
from src.api import DataAPI , Trainer
DataAPI.reconstruct_train_data()
Trainer.update_models()

predict is False , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 22.28 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 21.73 secs
 --> labels blocks merging (2)...... finished! Cost 4.75 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 114.89 secs
y blocks loading finished! Cost 269.22 secs
y blocks process...... finished! Cost 128.26 secs
y blocks masking...... finished! Cost 1.38 secs
y blocks saving ...... finished! Cost 2.88 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 402.61 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 33.11 secs
day blocks loading finished! Cost 33.57 secs
day blocks process...... finished! Cost 5.12 secs
day blocks masking...... finished! Cost 1.16 secs


In [ ]:
from src.api import Trainer
Trainer.update_models()

Use device name: NVIDIA GeForce RTX 4090


24-06-24 01:10:43|MOD:display     |: Model Specifics:
24-06-24 01:10:43|MOD:display     |: Start Process [Data] at Mon Jun 24 01:10:43 2024!


--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gru_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=5,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gruRES_day',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_types': ['best', 'swalast', '

24-06-24 01:10:51|MOD:display     |: Finish Process [Data], Cost 7.9 Secs
24-06-24 01:10:51|MOD:display     |: Start Process [Fit] at Mon Jun 24 01:10:51 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


...

FirstBite Ep#  0 : loss  1.00382, train-0.00516, valid-0.03649, best-0.0365, lr1.3e-03
FirstBite Ep#  5 : loss  0.86311, train 0.13764, valid 0.09228, best 0.0923, lr2.5e-03
FirstBite Ep# 10 : loss  0.84547, train 0.15544, valid 0.09630, best 0.0963, lr1.9e-03
FirstBite Ep# 15 : loss  0.83038, train 0.17066, valid 0.08826, best 0.0963, lr1.0e-07
FirstBite Ep# 20 : loss  0.82732, train 0.17363, valid 0.08575, best 0.0963, lr9.4e-04
FirstBite Ep# 25 : loss  0.81729, train 0.18370, valid 0.08407, best 0.0963, lr3.1e-04
FirstBite Ep# 30 : loss  0.81421, train 0.18647, valid 0.08262, best 0.0963, lr1.6e-04
24-06-24 01:27:46|MOD:display     |: gruRTN_day #1 @20231201|FirstBite Ep# 31 EarlyStop|Train 0.1865 Valid 0.0826 BestVal 0.0826|Cost  1.9Min,  3.6Sec/Ep
FirstBite Ep#  0 : loss  1.00293, train-0.00341, valid-0.00344, best-0.0034, lr1.3e-03
FirstBite Ep#  5 : loss  0.87709, train 0.12551, valid 0.08796, best 0.1129, lr2.5e-03
FirstBite Ep# 10 : loss  0.86180, train 0.13731, valid 0.10436,

--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gruRES_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=5,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gruRES_day',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_types': ['best', 'swalast'

24-06-24 01:36:34|MOD:display     |: Finish Process [Data], Cost 2.0 Secs
24-06-24 01:36:34|MOD:display     |: Start Process [Fit] at Mon Jun 24 01:36:34 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


FirstBite Ep#  0 : loss  1.01252, train-0.01521, valid-0.03742, best-0.0374, lr1.3e-03
FirstBite Ep#  5 : loss  0.91026, train 0.08135, valid 0.03185, best 0.0599, lr2.5e-03
FirstBite Ep# 10 : loss  0.89291, train 0.09436, valid 0.04522, best 0.0599, lr1.9e-03
FirstBite Ep# 15 : loss  0.87755, train 0.10648, valid 0.03959, best 0.0599, lr1.0e-07
FirstBite Ep# 20 : loss  0.87426, train 0.10831, valid 0.03543, best 0.0599, lr9.4e-04
FirstBite Ep# 21 : loss  0.87082, train 0.11088, valid 0.03980, best 0.0599, lr6.3e-04, Next attempt goes!
Retrain#1 Ep#  0 : loss  0.99637, train 0.00462, valid 0.03106, best 0.0311, lr1.3e-04
Retrain#1 Ep#  5 : loss  0.94196, train 0.05698, valid 0.03578, best 0.0560, lr2.5e-04
Retrain#1 Ep# 10 : loss  0.92553, train 0.06750, valid 0.03080, best 0.0560, lr1.9e-04
Retrain#1 Ep# 15 : loss  0.91653, train 0.07499, valid 0.03537, best 0.0560, lr1.0e-07
Retrain#1 Ep# 20 : loss  0.91509, train 0.07561, valid 0.04185, best 0.0560, lr9.4e-05
Retrain#1 Ep# 21 : loss